In [1]:
from google.colab import drive

drive.mount('/content/drive', force_remount=True)
repository = 'evaluating_factuality_word_definitions'

%cd /content/drive/My Drive/{repository}

Mounted at /content/drive
/content/drive/My Drive/evaluating_factuality_word_definitions


In [2]:
%%capture
!pip install datasets
!pip install peft
!pip install rank_bm25
!pip install einops

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 9.3 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 13.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-

In [3]:
%%capture
!python -m spacy download en_core_web_lg
!python -m spacy download de_core_news_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 2.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 567.8/567.8 MB 2.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('de_core_news_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [1]:
from datasets import Dataset
from dataset.def_dataset import DefinitionDataset, Fact
from config import DB_URL
from transformers import AutoTokenizer
from models.evidence_selection_model import EvidenceSelectionModel
import torch
from general_utils.fever_scorer import fever_score
from pipeline.pipeline import TestPipeline, WikiPipeline
from general_utils.utils import build_fever_instance
from general_utils.utils import convert_document_id_to_word
from sklearn.metrics import classification_report
from tqdm import tqdm

In [14]:
dataset_query = """
with unique_claims as (
select distinct dd.id, dd.claim, dd.label, dd.evidence_wiki_url, dd.set_type
from def_dataset dd)
select uq.id, uq.claim, uq.label, docs.document_id, docs.text,
       docs.lines, se.evidence_lines as evidence_lines, GROUP_CONCAT(af.fact, '--;--') as atomic_facts
from unique_claims as uq
    join selected_evidence se on uq.id = se.claim_id
    join documents docs on docs.document_id = uq.evidence_wiki_url
    left join atomic_facts af on af.claim_id = uq.id
where uq.set_type = 'dev' --'{set_type}' and 10=10
group by uq.id, docs.document_id
"""

raw_dataset = Dataset.from_sql(dataset_query.format(set_type='dev'), con=DB_URL)
dataset = DefinitionDataset(raw_dataset, tokenizer=None, model='claim_verification')

Generating train split: 0 examples [00:00, ? examples/s]

Filter:   0%|          | 0/2551 [00:00<?, ? examples/s]

# Training

In [32]:
from transformers import AutoModel

device = "cuda" if torch.cuda.is_available() else "cpu"

#selection_model_tokenizer = AutoTokenizer.from_pretrained('Snowflake/snowflake-arctic-embed-m-long')
#model = AutoModel.from_pretrained('Snowflake/snowflake-arctic-embed-m-long', trust_remote_code=True, add_pooling_layer=False, safe_serialization=True)
selection_model_tokenizer = AutoTokenizer.from_pretrained('lukasellinger/evidence_selection_model-v2')
model = AutoModel.from_pretrained('lukasellinger/evidence_selection_model-v2', trust_remote_code=True, add_pooling_layer=False, safe_serialization=True)
selection_model = EvidenceSelectionModel(model).to(device)

# still using base
verification_model=None
verification_model_tokenizer=None

tokenizer_config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

...

A new version of the following files was downloaded from https://huggingface.co/lukasellinger/evidence_selection_model-v2:
- configuration_hf_nomic_bert.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_hf_nomic_bert.py:   0%|          | 0.00/52.6k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/lukasellinger/evidence_selection_model-v2:
- modeling_hf_nomic_bert.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


model.safetensors:   0%|          | 0.00/547M [00:00<?, ?B/s]

<All keys matched successfully>


In [23]:
dataset_query = """
with unique_claims as (
select distinct dd.id, dd.short_claim as claim, dd.label, dd.evidence_wiki_url, dd.set_type
from def_dataset dd)
select uq.id, uq.claim, uq.label, docs.document_id, docs.text,
       docs.lines, se.evidence_lines as evidence_lines, GROUP_CONCAT(af.fact, '--;--') as atomic_facts
from unique_claims as uq
    join selected_evidence se on uq.id = se.claim_id
    join documents docs on docs.document_id = uq.evidence_wiki_url
    left join atomic_facts_fever_short_dissim af on af.claim_id = uq.id
where uq.set_type = '{set_type}' and 10=10
group by uq.id, docs.document_id
"""

raw_dataset = Dataset.from_sql(dataset_query.format(set_type='dev'), con=DB_URL)
dataset = DefinitionDataset(raw_dataset, tokenizer=None, model='claim_verification')

Generating train split: 0 examples [00:00, ? examples/s]

Filter:   0%|          | 0/2551 [00:00<?, ? examples/s]

In [24]:
pipeline = TestPipeline(selection_model=selection_model, selection_model_tokenizer=selection_model_tokenizer, word_lang='de')

pr_labels = []
gt_labels = []
factualities = []
not_in_wiki = 0
for entry in tqdm(dataset):
    word = entry.get('document_id')
    fallback_word = convert_document_id_to_word(word)
    claim = entry['claim']
    atomic_facts = entry['atomic_facts']
    atomic_facts = atomic_facts.split('--;--') if atomic_facts else []
    factuality = pipeline.verify(word, claim, fallback_word, only_intro=True, split_facts=False, atomic_claims=atomic_facts)
    factualities.append(factuality)
    if factuality.get('factuality') == 1:
        pr_labels.append(Fact.SUPPORTED.to_factuality())
    elif factuality.get('factuality') == -1:
        not_in_wiki += 1
        continue
    else:
        pr_labels.append(Fact.NOT_SUPPORTED.to_factuality())
    gt_labels.append(1) if entry['label'] == 'SUPPORTS' else gt_labels.append(0)

print(f'Not in wiki {not_in_wiki}')
print(classification_report(gt_labels, pr_labels, zero_division=0))

100%|██████████| 2551/2551 [31:54<00:00,  1.33it/s]  

Not in wiki 0
              precision    recall  f1-score   support

           0       0.97      0.91      0.94      1760
           1       0.82      0.93      0.87       791

    accuracy                           0.92      2551
   macro avg       0.89      0.92      0.90      2551
weighted avg       0.92      0.92      0.92      2551



In [27]:

dataset_query = """
with unique_claims as (
select distinct dd.id, dd.short_claim as claim, dd.label, dd.evidence_wiki_url, dd.set_type
from def_dataset dd)
select uq.id, uq.claim, uq.label, docs.document_id, docs.text,
       docs.lines, se.evidence_lines as evidence_lines, GROUP_CONCAT(af.fact, '--;--') as atomic_facts
from unique_claims as uq
    join selected_evidence se on uq.id = se.claim_id
    join documents docs on docs.document_id = uq.evidence_wiki_url
    left join atomic_facts_fever_short_dissim af on af.claim_id = uq.id
where uq.set_type = '{set_type}' and 15=15
group by uq.id, docs.document_id
"""

raw_dataset = Dataset.from_sql(dataset_query.format(set_type='dev'), con=DB_URL)
dataset = DefinitionDataset(raw_dataset, tokenizer=None, model='claim_verification')

In [31]:
## Short claims wiki
pipeline = WikiPipeline(selection_model=selection_model, selection_model_tokenizer=selection_model_tokenizer, word_lang='de')

pr_labels = []
gt_labels = []
factualities = []
not_in_wiki = 0
for entry in tqdm(dataset):
    word = entry.get('document_id')
    fallback_word = convert_document_id_to_word(word)
    claim = entry['claim']
    atomic_facts = entry['atomic_facts']
    atomic_facts = atomic_facts.split('--;--') if atomic_facts else []
    factuality = pipeline.verify(fallback_word, claim, fallback_word, only_intro=True, split_facts=False, atomic_claims=atomic_facts)
    factualities.append(factuality)
    if factuality.get('factuality') == 1:
        pr_labels.append(Fact.SUPPORTED.to_factuality())
    elif factuality.get('factuality') == -1:
        not_in_wiki += 1
        continue
    else:
        pr_labels.append(Fact.NOT_SUPPORTED.to_factuality())
    gt_labels.append(1) if entry['label'] == 'SUPPORTS' else gt_labels.append(0)

print(f'Not in wiki {not_in_wiki}')
print(classification_report(gt_labels, pr_labels, zero_division=0))

100%|██████████| 2551/2551 [1:03:03<00:00,  1.48s/it]

Not in wiki 1235
              precision    recall  f1-score   support

           0       0.88      0.90      0.89       904
           1       0.76      0.73      0.74       412

    accuracy                           0.84      1316
   macro avg       0.82      0.81      0.82      1316
weighted avg       0.84      0.84      0.84      1316



In [30]:
print(classification_report(gt_labels, pr_labels, zero_division=0))


              precision    recall  f1-score   support

           0       0.88      0.89      0.88       522
           1       0.77      0.75      0.76       253

    accuracy                           0.84       775
   macro avg       0.82      0.82      0.82       775
weighted avg       0.84      0.84      0.84       775



In [ ]:
## claims test

dataset_query = """
with unique_claims as (
select distinct dd.id, dd.claim, dd.label, dd.evidence_wiki_url, dd.set_type
from def_dataset dd)
select uq.id, uq.claim, uq.label, docs.document_id, docs.text,
       docs.lines, se.evidence_lines as evidence_lines, GROUP_CONCAT(af.fact, '--;--') as atomic_facts
from unique_claims as uq
    join selected_evidence se on uq.id = se.claim_id
    join documents docs on docs.document_id = uq.evidence_wiki_url
    left join atomic_facts_fever_dissim af on af.claim_id = uq.id
where uq.set_type = 'dev' --'{set_type}' and 10=10
group by uq.id, docs.document_id
"""

raw_dataset = Dataset.from_sql(dataset_query.format(set_type='dev'), con=DB_URL)
dataset = DefinitionDataset(raw_dataset, tokenizer=None, model='claim_verification')

pipeline = TestPipeline(selection_model=selection_model, selection_model_tokenizer=selection_model_tokenizer, word_lang='de')

pr_labels = []
gt_labels = []
factualities = []
not_in_wiki = 0
for entry in tqdm(dataset):
    word = entry.get('document_id')
    fallback_word = convert_document_id_to_word(word)
    claim = entry['claim']
    atomic_facts = entry['atomic_facts']
    atomic_facts = atomic_facts.split('--;--') if atomic_facts else []
    factuality = pipeline.verify(word, claim, fallback_word, only_intro=True, split_facts=False, atomic_claims=atomic_facts)
    factualities.append(factuality)
    if factuality.get('factuality') == 1:
        pr_labels.append(Fact.SUPPORTED.to_factuality())
    elif factuality.get('factuality') == -1:
        not_in_wiki += 1
        continue
    else:
        pr_labels.append(Fact.NOT_SUPPORTED.to_factuality())
    gt_labels.append(1) if entry['label'] == 'SUPPORTS' else gt_labels.append(0)

print(f'Not in wiki {not_in_wiki}')
print(classification_report(gt_labels, pr_labels, zero_division=0))

In [ ]:
## claims wiki

dataset_query = """
with unique_claims as (
select distinct dd.id, dd.claim, dd.label, dd.evidence_wiki_url, dd.set_type
from def_dataset dd)
select uq.id, uq.claim, uq.label, docs.document_id, docs.text,
       docs.lines, se.evidence_lines as evidence_lines, GROUP_CONCAT(af.fact, '--;--') as atomic_facts
from unique_claims as uq
    join selected_evidence se on uq.id = se.claim_id
    join documents docs on docs.document_id = uq.evidence_wiki_url
    left join atomic_facts_fever_dissim af on af.claim_id = uq.id
where uq.set_type = 'dev' --'{set_type}' and 10=10
group by uq.id, docs.document_id
"""

raw_dataset = Dataset.from_sql(dataset_query.format(set_type='dev'), con=DB_URL)
dataset = DefinitionDataset(raw_dataset, tokenizer=None, model='claim_verification')

pipeline = WikiPipeline(selection_model=selection_model, selection_model_tokenizer=selection_model_tokenizer, word_lang='de')

pr_labels = []
gt_labels = []
factualities = []
not_in_wiki = 0
for entry in tqdm(dataset):
    word = entry.get('document_id')
    fallback_word = convert_document_id_to_word(word)
    claim = entry['claim']
    atomic_facts = entry['atomic_facts']
    atomic_facts = atomic_facts.split('--;--') if atomic_facts else []
    factuality = pipeline.verify(fallback_word, claim, fallback_word, only_intro=True, split_facts=False, atomic_claims=atomic_facts)
    factualities.append(factuality)
    if factuality.get('factuality') == 1:
        pr_labels.append(Fact.SUPPORTED.to_factuality())
    elif factuality.get('factuality') == -1:
        not_in_wiki += 1
        continue
    else:
        pr_labels.append(Fact.NOT_SUPPORTED.to_factuality())
    gt_labels.append(1) if entry['label'] == 'SUPPORTS' else gt_labels.append(0)

print(f'Not in wiki {not_in_wiki}')
print(classification_report(gt_labels, pr_labels, zero_division=0))

In [32]:
test_pipeline = TestPipeline(selection_model=selection_model,selection_model_tokenizer=selection_model_tokenizer)

#test_pipeline = TestPipeline()

pr_labels = []
gt_labels = []
fever_instances = []
for entry in tqdm(dataset):
    
    output = test_pipeline.verify(entry['document_id'], entry['claim'], split_facts=True)
    pr_labels.extend([fact.to_factuality() for fact in output['factualities']])
    
    if entry['label'] == 'SUPPORTS':
        gt_labels += [Fact.SUPPORTED.to_factuality()] * len(output['factualities'])
    else:
        gt_labels += [Fact.NOT_SUPPORTED.to_factuality()] * len(output['factualities'])
        
    evidence = entry['evidence_lines'].split(';')
    predicted_label = output.get('factualities')[0]  # TODO add atomic fact support
    predicted_evidence = output.get('evidences')
    fever_instance = build_fever_instance(entry['label'], evidence, entry['document_id'], predicted_label, predicted_evidence)
    fever_instances.append(fever_instance)

print(classification_report(gt_labels, pr_labels, zero_division=0))
strict_score, label_accuracy, precision, recall, f1 = fever_score(fever_instances)

print(strict_score)
print(label_accuracy)
print(precision)  # TP / TP + FP not too important, rather at least one TP than none
print(recall)     # more important
print(f1)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
100%|██████████| 2313/2313 [03:07<00:00, 12.33it/s]

              precision    recall  f1-score   support

          -1       0.00      0.00      0.00         0
           0       0.97      0.87      0.92      1238
           1       0.97      0.93      0.95      1075

    accuracy                           0.90      2313
   macro avg       0.64      0.60      0.62      2313
weighted avg       0.97      0.90      0.93      2313

0.8802421098140942
0.8996973627323822
0.42080991497334974
0.9684392563769996
0.5866893420375288


In [ ]:
print([int(label) for label in pr_labels])
print(gt_labels)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, -1, -1, -1, 1, 1, 0, 1, -1, 1, 0, -1, -1, -1, -1]
[1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0]


In [33]:
strict_score, label_accuracy, precision, recall, f1 = fever_score(fever_instances, use_gold_labels=True)
print(strict_score)
print(label_accuracy)
print(precision)
print(recall)
print(f1)

0.9684392563769996
1.0
0.42080991497334974
0.9684392563769996
0.5866893420375288


In [ ]:
pipeline = WikiPipeline(selection_model=selection_model, selection_model_tokenizer=selection_model_tokenizer)

pr_labels = []
gt_labels = []
for entry in tqdm(dataset):
    word = convert_document_id_to_word(entry['document_id'])

    factuality = pipeline.verify(word, entry['claim'])
    pr_labels.extend([fact.to_factuality() for fact in factuality])
    gt_labels += [Fact[entry['label']].to_factuality()] * len(factuality)

print(classification_report(gt_labels, pr_labels, zero_division=0))
print(classification_report(gt_labels, pr_labels, zero_division=0))
strict_score, label_accuracy, precision, recall, f1 = fever_score(fever_instances)

print(strict_score)
print(label_accuracy)
print(precision)
print(recall)
print(f1)

  0%|          | 0/20 [00:04<?, ?it/s]


AttributeError: 'str' object has no attribute 'to_factuality'

In [ ]:
print([int(label) for label in pr_labels])
print(gt_labels)

# German DPR test

In [33]:
from datasets import load_dataset

dataset = load_dataset("lukasellinger/german_dpr_claim_verification_dissim-v1")
#dataset = load_dataset("lukasellinger/german_dpr_claim_verification_mixtral-v1")
#dataset = load_dataset("lukasellinger/german_claim_verification_atomic_jan-v1")
#dataset = dataset.filter(lambda x: x['id'] == 75)
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['id', 'question', 'claim', 'english_claim', 'fact', 'word', 'english_word', 'context', 'label', 'atomic_facts'],
        num_rows: 168
    })
})


In [34]:
pipeline = WikiPipeline(selection_model=selection_model, selection_model_tokenizer=selection_model_tokenizer, word_lang='de')

pr_labels = []
gt_labels = []
factualities = []
not_in_wiki = 0
for entry in tqdm(dataset.get('train')):
    word = entry.get('word')
    english_word = entry.get('english_word')
    claim = entry['english_claim']
    atomic_facts = entry['atomic_facts']
    atomic_facts = atomic_facts.split('--;--') if atomic_facts else []
    factuality = pipeline.verify(word, claim, english_word, only_intro=True, split_facts=False, atomic_claims=atomic_facts)
    factualities.append(factuality)
    if factuality.get('factuality') == 1:
        pr_labels.append(Fact.SUPPORTED.to_factuality())
    elif factuality.get('factuality') == -1:
        not_in_wiki += 1
        continue
    else:
        pr_labels.append(Fact.NOT_SUPPORTED.to_factuality())
    gt_labels.append(Fact[entry['label']].to_factuality())

print(f'Not in wiki {not_in_wiki}')
print(classification_report(gt_labels, pr_labels, zero_division=0))

100%|██████████| 168/168 [10:34<00:00,  3.78s/it]

Not in wiki 36
              precision    recall  f1-score   support

           0       0.66      0.94      0.78        66
           1       0.89      0.52      0.65        66

    accuracy                           0.73       132
   macro avg       0.78      0.73      0.71       132
weighted avg       0.78      0.73      0.71       132



In [21]:
print(not_in_wiki)
print(classification_report(gt_labels, pr_labels, zero_division=0))

1
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       1.00      0.71      0.83         7

    accuracy                           0.71         7
   macro avg       0.50      0.36      0.42         7
weighted avg       1.00      0.71      0.83         7



In [5]:
factualities_own_selection = factualities
print(pr_labels)
print(gt_labels)

[1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [15]:
print(dataset.get('train')[194])
print(len(pr_labels))

{'id': 3399, 'word': 'pulmologisch', 'label': 'SUPPORTED', 'claim': 'lungenfachärztlich, lungenärztlich', 'context_sentence': '„Pulmologisch bedingte Thoraxschmerzen treten v.a. Bei Erkrankungen der Pleura auf […].“ „Die pulmologischen Leitsymptome sind: Dyspnoe, Husten, Auswurf, Hämoptoe und Thoraxschmerz.“', 'english_claim': 'specialist in lung medicine, specialist in lung medicine', 'english_word': 'Pulmologically', 'atomic_facts': 'Pulmology is a branch of medicine.--;--Pulmology specializes in lung medicine.'}
194


In [24]:
pr_labels2 = [1 if factuality.get('factuality') > 0 else 0 for factuality in factualities if factuality.get('factuality') != -1]

In [23]:
print(gt_labels[7])

1


In [25]:
print(classification_report(gt_labels, pr_labels2, zero_division=0))

              precision    recall  f1-score   support

           0       0.71      0.91      0.79        66
           1       0.87      0.62      0.73        66

    accuracy                           0.77       132
   macro avg       0.79      0.77      0.76       132
weighted avg       0.79      0.77      0.76       132

